In [1]:
!pip install -q efficientnet --quiet

In [2]:
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import gc
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pickle
from tensorflow.keras.callbacks import CSVLogger
import time
from efficientnet.tfkeras import center_crop_and_resize
from tensorflow_addons.losses import TripletSemiHardLoss
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from kaggle_datasets import KaggleDatasets

In [3]:
from model_semantic import MODEL_INPUT, linear_warmup, build_model_extractor, timecallback, Score_call

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)


Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


In [5]:
dataset_loader_path = '../input/dataset-loader-25/'

In [6]:
MODEL_TYPE = 'B5'
MODEL_ = 'efficientnet'

IMAGE_SIZE = MODEL_INPUT[MODEL_TYPE]
BATCH_SIZE = 256 * strategy.num_replicas_in_sync
EPOCHS = 1
NUMBER_OF_IMAGE = 16
CLASS_PER_BATCH = BATCH_SIZE // NUMBER_OF_IMAGE
EFF_NET_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

# # Data access
GCS_DS_PATH = KaggleDatasets().get_gcs_path('landmark-retrieval-2020')

In [7]:
with open(dataset_loader_path + 'train_.pkl', 'rb') as f:
    train_ = pickle.load(f)
    
train_['train_path'] = [GCS_DS_PATH + x[32:] for x in tqdm(train_.train_path)]

N_CLASS = train_.landmark_id.unique().shape[0]
IMAGE_PER_BATCH = N_CLASS * NUMBER_OF_IMAGE
STEPS_PER_EPOCH = N_CLASS // CLASS_PER_BATCH

In [8]:
def land_generator(dataset = train_, number_of_image = 4):
    
    landmark_ids = dataset["landmark_id"].unique()
    dict_landmark_to_images_mapping = dataset.groupby("landmark_id")["id"].apply(list).to_dict()
    path_dict = dataset.set_index("id")["train_path"].to_dict()
    indexes = landmark_ids.copy()
    np.random.shuffle(indexes)

    while True:
        
        for land in indexes:
            all_landmark_image = dict_landmark_to_images_mapping[land]
            images_id = np.random.choice(all_landmark_image, number_of_image, replace = False)
            
            for images in images_id:
                image_path = path_dict[images]
                yield image_path, land

In [9]:
path_ = []
label_ = []

for epoch in range(EPOCHS):
    gen = land_generator()
    for it in tqdm(range(IMAGE_PER_BATCH)):
        image_path, land = next(gen)
        path_.append(image_path)
        label_.append(land)

In [10]:
slicer = pd.DataFrame({'path': path_, 'label': label_})

In [11]:
def decode_image(filename, label, image_size = IMAGE_SIZE, crop_padding = 32):
    #image bytes
    image_bytes = tf.io.read_file(filename)
    
    #extract shape and get crop window
    shape = tf.image.extract_jpeg_shape(image_bytes)
    image_height = shape[0]
    image_width = shape[1]

    padded_center_crop_size = tf.cast(
      ((image_size / (image_size + crop_padding)) *
       tf.cast(tf.minimum(image_height, image_width), tf.float32)),
      tf.int32)

    offset_height = ((image_height - padded_center_crop_size) + 1) // 2
    offset_width = ((image_width - padded_center_crop_size) + 1) // 2
    crop_window = tf.stack([offset_height, offset_width,
                          padded_center_crop_size, padded_center_crop_size])
    
    #decode and crop image --> resize
    image = tf.image.decode_and_crop_jpeg(image_bytes, crop_window, channels=3)
    image = tf.image.resize([image], [image_size, image_size], method = tf.image.ResizeMethod.BICUBIC)[0]

    image = tf.reshape(image, [image_size, image_size, 3])
    image = tf.image.convert_image_dtype(image, dtype = tf.float32)
    
    #ensure pixel are [0, 255]
    image = tf.clip_by_value(image, 0.0, 255.0)
    image = tf.cast(image, dtype = tf.float32)
    
    #imagenet pre process
    image = preprocess_input(image, mode = 'torch')
    
    return(image, label)



In [12]:
training_generator = (
    tf.data.Dataset.
    from_tensor_slices((slicer['path'], slicer['label']))
    .map(decode_image, num_parallel_calls = AUTO)
    .repeat(EPOCHS)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

ignore_order = tf.data.Options()
ignore_order.experimental_deterministic = True # disable order, increase speed

training_generator = training_generator.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order

In [13]:
total_step = EPOCHS * STEPS_PER_EPOCH

initial_lr = 0.001
num_warmup_steps = 61# (STEPS_PER_EPOCH * EPOCHS) //10

optimizer_warmup = linear_warmup(init_lr = initial_lr, num_train_steps = total_step, num_warmup_steps = num_warmup_steps)

In [14]:
with strategy.scope():
    model = build_model_extractor(
        optimizer_warmup, EFF_NET_SHAPE, TripletSemiHardLoss(),
        model = MODEL_, modeltype = 'B5', weights = 'noisy-student', trainable = False)

115261440/115255328 [==============================] - 1s 0us/step


In [15]:
model.load_weights('../input/effb5-block-extractor-4/efficientnetB5_embedding_model.hdf5')

In [16]:
for i in [2, 3, 4, 5, 6]:
    print('Gem {}; Value: {}'.format(i, model.get_layer(f'gem_block_{i}').p.numpy()[0]))

Gem 2; Value: 3.101045846939087
Gem 3; Value: 3.0898396968841553
Gem 4; Value: 3.220949172973633
Gem 5; Value: 2.49170184135437
Gem 6; Value: 1.5477491617202759


In [17]:
history = model.fit(
    training_generator,
    batch_size=BATCH_SIZE, epochs=EPOCHS,
    steps_per_epoch = STEPS_PER_EPOCH,
    verbose = 1
)

120/120 [==============================] - 8732s 73s/step - loss: 0.6753


In [18]:
for i in [2, 3, 4, 5, 6]:
    print('Gem {}; Value: {}'.format(i, model.get_layer(f'gem_block_{i}').p.numpy()[0]))

Gem 2; Value: 3.103010416030884
Gem 3; Value: 3.0975868701934814
Gem 4; Value: 3.2258481979370117
Gem 5; Value: 2.5000078678131104
Gem 6; Value: 1.5563220977783203


In [19]:
model.save_weights(f'{MODEL_}{MODEL_TYPE}_embedding_model.hdf5')

In [20]:
!mkdir -p saved_model
model.save('saved_model/my_model')

UnimplementedError: File system scheme '[local]' not implemented (file: 'saved_model/my_model/variables/variables_temp_ec94055740ce455bbd1395ec7ab2b333/part-00000-of-00002')
	Encountered when executing an operation using EagerExecutor. This error cancels all future operations and poisons their output tensors. [Op:Identity]